<a href="https://colab.research.google.com/github/Katonokatono/Suicide/blob/model/Transformers_Distilbert_Uncased_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import numpy as np    
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
! pip install transformers

In [3]:
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

In [4]:
! pip install pyyaml h5py

In [5]:
!pip install wandb
!wandb login 
import wandb
from wandb.keras import WandbCallback

wandb: Currently logged in as: kanoak (use `wandb login --relogin` to force relogin)


In [6]:
!pip install ktrain

In [7]:
#initialise wandb
#%env WANDB_PROJECT='gpt3', entity='alpha-coders'
#%env WANDB_LOG_MODEL=true

In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [9]:
import ktrain
from ktrain import text
import pandas as pd

In [10]:
data_df = pd.read_csv('/content/balanced_data.csv')

In [11]:
data_df.head()

,Unnamed: 0,Unnamed: 0.1,friends_count,followers_count,statuses_count,created_at,retweet_count,label,town,clean_text
0,2864,133,424,55,412,9/13/2021 1:47,0,thoughts,nyeri,go torture deserve get bully since yr old body...
1,3350,624,3438,1562,9614,9/9/2021 4:07,6,thoughts,majengo,remember suicide thought thought act get contr...
2,2864,133,424,55,412,9/13/2021 1:47,0,thoughts,nyeri,go torture deserve get bully since yr old body...
3,3151,422,277,98,729,9/10/2021 8:13,0,thoughts,nairobi,suicide squad exist worldsuicidepreventionday ...
4,3259,533,5486,48711,295173,9/10/2021 4:09,0,thoughts,nairobi,people rarely commit suicide want die want sto...


In [12]:
data_df['label'].value_counts()

awareness          355
anxiety            355
stress             355
general tweet      355
thoughts           355
depression         355
substance abuse    355
Name: label, dtype: int64

In [13]:
data_df.isnull().sum()

Unnamed: 0         0
Unnamed: 0.1       0
friends_count      0
followers_count    0
statuses_count     0
created_at         0
retweet_count      0
label              0
town               0
clean_text         0
dtype: int64

In [14]:
data_df.dropna(inplace=True)

In [15]:
data_df.isnull().sum()

Unnamed: 0         0
Unnamed: 0.1       0
friends_count      0
followers_count    0
statuses_count     0
created_at         0
retweet_count      0
label              0
town               0
clean_text         0
dtype: int64

In [16]:
# creating a dataframe with the tweet and target column
data_df= data_df[['clean_text','label']]
data_df.head()

,clean_text,label
0,go torture deserve get bully since yr old body...,thoughts
1,remember suicide thought thought act get contr...,thoughts
2,go torture deserve get bully since yr old body...,thoughts
3,suicide squad exist worldsuicidepreventionday ...,thoughts
4,people rarely commit suicide want die want sto...,thoughts


In [17]:
# Defining our X and Y varaibles
X = data_df['clean_text']
y= data_df['label']
print(X.head())
print('/n')
print(y.head())

0    go torture deserve get bully since yr old body...
1    remember suicide thought thought act get contr...
2    go torture deserve get bully since yr old body...
3    suicide squad exist worldsuicidepreventionday ...
4    people rarely commit suicide want die want sto...
Name: clean_text, dtype: object
/n
0    thoughts
1    thoughts
2    thoughts
3    thoughts
4    thoughts
Name: label, dtype: object


In [18]:
activity_count = y.value_counts()
activity_count

awareness          355
anxiety            355
stress             355
general tweet      355
thoughts           355
depression         355
substance abuse    355
Name: label, dtype: int64

In [19]:
# Splitting our data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# checking our classes
data_df.label.unique()

array(['thoughts', 'substance abuse', 'stress', 'general tweet',
       'anxiety', 'awareness', 'depression'], dtype=object)

In [21]:
# creating a list varaiable for our classes
target = ['general tweet', 'anxiety', 'stress', 'depression',
       'substance abuse', 'awareness', 'thoughts']

In [22]:
# Defining our X and Y varaibles
X_test = data_df['clean_text'].astype(str)
y_test = data_df['label'].astype(str)
print(X_test.head())
print('/n')
print(y_test.head())

0    go torture deserve get bully since yr old body...
1    remember suicide thought thought act get contr...
2    go torture deserve get bully since yr old body...
3    suicide squad exist worldsuicidepreventionday ...
4    people rarely commit suicide want die want sto...
Name: clean_text, dtype: object
/n
0    thoughts
1    thoughts
2    thoughts
3    thoughts
4    thoughts
Name: label, dtype: object


In [23]:
# converting our train varaibles to numpy array
X_train= X_train.to_numpy()
y_train= y_train.to_numpy()

In [24]:
# converting our test varaibles to numpy array
X_test= X_test.to_numpy()
y_test = y_test.to_numpy()

In [25]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

In [26]:
# defining our model object and preprocessing our data
MODEL_NAME = "distilbert-base-uncased"
t = text.Transformer(MODEL_NAME, maxlen=500, class_names= target)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


preprocessing train...
language: en
train sequence lengths:
	mean : 9
	95percentile : 14
	99percentile : 17


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:600: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 9
	95percentile : 14
	99percentile : 17


In [27]:
# Import required model class
from transformers import TFDistilBertForSequenceClassification
# Download pre-trained model
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, 
                                                              num_labels=7,
                                                              output_hidden_states=True, 
                                                              output_attentions=True
                                                              )

In [28]:
from transformers import DistilBertConfig
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512)
model = TFDistilBertForSequenceClassification(config)

In [29]:
#initialise wandb
wandb.init(project='gpt3', save_code= True, entity='Alpha-coders')



wandb: Currently logged in as: kanoak (use `wandb login --relogin` to force relogin)


In [30]:
from transformers import TFTrainer,TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs',            
    logging_steps=10,
    report_to="wandb",  # enable logging to W&B
    run_name="Augmented-trainset"
       
)
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
    


trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=trn,         
    eval_dataset=val             
)

In [51]:
learner.fit_onecycle(1e-5, 10, callbacks=[WandbCallback()])



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/10
332/332 [==============================] - 226s 679ms/step - loss: 1.0618 - accuracy: 0.6876 - val_loss: 0.9804 - val_accuracy: 0.7111


wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
332/332 [==============================] - 226s 679ms/step - loss: 0.9644 - accuracy: 0.7113 - val_loss: 0.8570 - val_accuracy: 0.7372
Epoch 3/10
332/332 [==============================] - 226s 679ms/step - loss: 0.8426 - accuracy: 0.7409 - val_loss: 0.7051 - val_accuracy: 0.7875
Epoch 4/10
332/332 [==============================] - 226s 678ms/step - loss: 0.6857 - accuracy: 0.7983 - val_loss: 0.5758 - val_accuracy: 0.8221
Epoch 5/10
332/332 [==============================] - 226s 678ms/step - loss: 0.5406 - accuracy: 0.8410 - val_loss: 0.4414 - val_accuracy: 0.8712
Epoch 6/10
332/332 [==============================] - 226s 679ms/step - loss: 0.4184 - accuracy: 0.8778 - val_loss: 0.3872 - val_accuracy: 0.8841
Epoch 7/10
332/332 [==============================] - 226s 679ms/step - loss: 0.3030 - accuracy: 0.9150 - val_loss: 0.3264 - val_accuracy: 0.9014
Epoch 8/10
332/332 [==============================] - 226s 679ms/step - loss: 0.2404 - accuracy: 0.9336 - val_loss: 0.3000 -

In [32]:
from tensorflow.keras.models import load_model

model.save('Aug_model')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [33]:
model = wandb.Artifact('learner', type='model')
model.add_file('Aug_model')

run.log_artifact(model)


ValueError: ignored

In [ ]:
#predictor.save('Model')

In [52]:
#predictor = ktrain.load_predictor('/tmp/toxic_detector')

In [53]:
wandb.finish

<function wandb.sdk.wandb_run.finish>

In [54]:
def finish(exit_code: int = None) -> None:
    """Marks a run as finished, and finishes uploading all data.
    This is used when creating multiple runs in the same process.
    We automatically call this method when your script exits.
    """
    if wandb.run:
        wandb.run.finish(exit_code=exit_code)

In [55]:
learner.validate(class_names=t.get_classes())

                 precision    recall  f1-score   support

        anxiety       0.87      0.88      0.88       355
      awareness       0.88      0.87      0.88       355
     depression       0.91      0.86      0.88       355
  general tweet       0.89      0.81      0.85       355
         stress       0.93      0.98      0.95       355
substance abuse       0.94      0.97      0.96       355
       thoughts       0.94      0.99      0.96       355

       accuracy                           0.91      2485
      macro avg       0.91      0.91      0.91      2485
   weighted avg       0.91      0.91      0.91      2485



array([[314,   7,  14,   8,  11,   1,   0],
       [  7, 310,  10,   9,   6,   7,   6],
       [ 18,  12, 305,   8,   5,   2,   5],
       [ 19,  19,   6, 286,   4,  10,  11],
       [  3,   0,   0,   3, 347,   2,   0],
       [  0,   2,   0,   4,   2, 346,   1],
       [  0,   1,   0,   2,   0,   0, 352]])

In [56]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

----------
id:694 | loss:6.63 | true:substance abuse | pred:awareness)



In [57]:
print(X_test[468])

sellyamutabi think high time care addiction alcohol recognize problem first


In [58]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [59]:
predictor.predict('would want people around depressed pearlradioke worldsuicidepreventionday cc')

'awareness'

In [67]:
# Calculate Recall score
from sklearn import metrics
import numpy as np
y_pred=predictor.predict(X_test)
print(metrics.recall_score(y_test,y_pred,average='macro'))

0.909456740442656


In [61]:
preds= predictor.predict_proba('would want people around depressed pearlradioke worldsuicidepreventionday cc')
preds

array([0.00504253, 0.91809136, 0.04895156, 0.01892317, 0.00371595,
       0.00244536, 0.00283   ], dtype=float32)

In [62]:
print([target], preds)
list_arrays= ([target], preds)

[['general tweet', 'anxiety', 'stress', 'depression', 'substance abuse', 'awareness', 'thoughts']] [0.00504253 0.91809136 0.04895156 0.01892317 0.00371595 0.00244536
 0.00283   ]


In [63]:
pd.DataFrame(list(map(np.ravel, list_arrays)))

,0,1,2,3,4,5,6
0,general tweet,anxiety,stress,depression,substance abuse,awareness,thoughts
1,0.00504253,0.918091,0.0489516,0.0189232,0.00371595,0.00244536,0.00283
